In [11]:
from preparator import Preparator 
import numpy as np 
from solver import Solver 
prep = Preparator()

from preparator import Preparator 
from logger import Logger 
import pandas as pd 
from datetime import datetime
import tqdm 
#Header
# Id (int),
# Valid (int),
# Updated (str),
# Matrix (mat),
# Size (int),
# Type (str),
# Rank (int),
# Density (flt),
# Stiffness (flt),
# Other (flt),
# Test_solver (str),
# Test (flt),
# Test_state (vec),
# Test_iterations (int),
# Test_time (flt),
# Train_solver (str),
# Preprocessing (str),
# Postprocessing (str),
# Train (flt),
# Train_state (vec),
# Train (flt), 
# Train_iterations_1 (int),
# Train_iterations_2 (int),
# Train_iterations_3 (int),
# Train_shots (int),
# Train_cost (flt),
# Train_time (flt),
# Flags (str)
# Baren (int)


In [12]:
# Prepare source dataset 

source_name = 'data/stiffness/Stiffness_size_{size}_stiffs_{stiffs}_stepsize_{stepsize}_shots_{shots}.csv'

Config = {
    'size' : 10,
    'stiffs' : [1e0, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6],
    'stiff_count' : 50, 
    'shots' : 1000, 
    'stepsize' : 0.25, 
    'accepterr': [0.01,0.05,0.01], 
    'iterations_limit' : 1, 
    'baren_err' : 0.01,
    'baren_threshold': 10
}

template = pd.read_csv('data/template.csv')

log = Logger (log_template = template, log_path = source_name.format(size = Config['size'], stiffs = Config['stiffs'], stepsize = Config['stepsize'], shots = Config['shots']))
exists = log.create_logfile()

log_file, id = log._load_logfile()


In [13]:

for stiff in Config['stiffs']: 
    for i in tqdm.tqdm(range(Config['stiff_count']), desc = f"Generating matrices with stiffness {stiff}"):
        
        id += 1
        matrix = prep.stiffnessPreparator(size = Config['size'], stiffness = stiff)
        current_datetime = datetime.now()
        data = { 
            'Id (int)' : id, 
            'Valid (int)' : 1, 
            'Updated (str)' : current_datetime.strftime("%Y-%m-%d %H:%M:%S"),
            'Matrix (mat)' : matrix,
            'Size (int)' : Config['size'],
            'Type (str)' : 'Stiff',
            'Rank (int)' : -1,
            'Density (flt)' : -1.0,
            'Stiffness (flt)' : stiff,
            'Penalty (flt)' : -1.0,
            'Other (flt)' : -1.0,
            'Test_solver (str)' : '-',
            'Test (flt)' : -1.0,
            'Test_state (vec)' : [],
            'Test_iterations (int)' : -1,
            'Test_time (flt)' : -1.0,
            'Train_solver (str)' : '-',
            'Preprocessing (str)' : '-',
            'Postprocessing (str)' : '-',
            'Train (flt)' : -1.0,
            'Train_state (vec)' : [],
            'Train_iterations (int)' : -1, 
            'Train_iterations_1 (int)' : -1,
            'Train_iterations_2 (int)' : -1,
            'Train_iterations_3 (int)' : -1,
            'Train_shots (int)': -1, 
            'Train_cost (flt)' : -1.0,
            'Train_time (flt)' : -1.0, 
            'Flags (str)' : '',
            'Baren (int)' : -1
        }
        
        log.log(data)

    
display(log.read(1))

Generating matrices with stiffness 1.0:  16%|█▌        | 8/50 [00:00<00:00, 76.91it/s]

Generating matrices with stiffness 1000000.0: 100%|██████████| 50/50 [00:02<00:00, 20.45it/s]


{'Id (int)': 1,
 'Valid (int)': 1,
 'Updated (str)': '2025-08-22 15:15:03',
 'Matrix (mat)': array([[-0.24727094,  0.2533453 ,  0.59447229,  0.55778235, -0.09260844,
         -0.16463568, -0.06952738,  0.08534268,  0.69904006,  0.52575072],
        [ 0.2533453 ,  0.26058783, -0.37869269, -1.        ,  0.08941295,
         -0.44284814, -0.42490131, -0.02336306,  0.43996261, -0.31425292],
        [ 0.59447229, -0.37869269, -0.1392754 , -0.06881384,  0.22219305,
         -0.54245159,  0.18825739,  0.56021424, -0.16916049,  0.83126575],
        [ 0.55778235, -1.        , -0.06881384,  0.17231462,  0.7924166 ,
         -0.12146863,  0.29525374,  0.10139351,  0.07816002, -0.15289818],
        [-0.09260844,  0.08941295,  0.22219305,  0.7924166 , -0.47217285,
         -0.09757695, -0.03092459,  0.56713787, -0.45908873,  0.11332089],
        [-0.16463568, -0.44284814, -0.54245159, -0.12146863, -0.09757695,
          0.40090203, -0.66052751,  0.30391856, -0.50339974, -0.35509725],
        [-0.06

In [16]:
import pennylane as qml 
testdev = qml.device ('default.qubit', wires = Config['size'], shots = Config['shots'])


In [17]:

def test (ids : np.ndarray, solver : str = 'bruteforce', log_data : bool = False, progressbar : bool = False ):

    ''' 
    ids : np.ndarray 
        a list of ids to compute 
    solver : str 
        solver to compute with 
    ''' 
    
    tester = Solver(dev = testdev, ansatz = '')

    if (progressbar):
        id_range = tqdm(ids, desc = f"Testing matrices with ids {min(ids)} <--> {max(ids)}") 
    else: 
        id_range = ids
    processed_data = []
    for id in id_range:

        data = log.read(id)

        print(data)
        def test_solver(Q : np.ndarray, solver : str = 'bruteforce'): 
            ''' 
            tester wrapped
            '''

            match solver: 
                case 'bruteforce':
                    J, h = prep.isingForm(Q)
                    return tester.checkIsing(J, h)
                case 'GW':
                    pass
                case 'Annealing': 
                    pass
            
        Q = data['Matrix (mat)']

        tested_data = data.copy()
        
        test, test_state, test_time = test_solver(Q)
        current_datetime = datetime.now()

        tested_data['Test (flt)'] = test 
        tested_data['Test_state (vec)'] = test_state[0].numpy()
        tested_data['Test_time (flt)'] = test_time
        tested_data['Test_solver (flt)'] = solver
        tested_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")

        processed_data.append(tested_data)

        if (log_data): 
            log.log(tested_data, byid = True, Id = id)
    return processed_data


In [ ]:
# test(range(1,21), solver = 'bruteforce')

Columns in data/stiffness/Stiffness_size_10_stiffs_[1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0, 1000000.0]_stepsize_0.25_shots_1000.csv are broken. Expected Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Penalty (flt)', 'Other (flt)', 'Test_solver (str)',
       'Test (flt)', 'Test_state (vec)', 'Test_iterations (int)',
       'Test_time (flt)', 'Train_solver (str)', 'Preprocessing (str)',
       'Postprocessing (str)', 'Train (flt)', 'Train_state (vec)',
       'Train_iterations (int)', 'Train_iterations_1 (int)',
       'Train_iterations_2 (int)', 'Train_iterations_3 (int)',
       'Train_cost (flt)', 'Train_shots (int)', 'Train_time (flt)',
       'Flags (str)', 'Baren (int)'],
      dtype='object') and got Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Other 

KeyboardInterrupt: 

In [18]:


ids = range (1,351)
cores = 70
poolsize = len(ids)//cores

from multiprocessing import Pool
from functools import partial

if (__name__ == '__main__') and (len(ids) % poolsize == 0): 
    
    solvers = [] 

    ids = np.array(ids) 
    index = 0
    pool_ids = ids.reshape(len(ids) // poolsize, poolsize)
    display(pool_ids)
    
    args = []
    for i in range (len(pool_ids)):
        if (Config['size'] > 12):
            args.append((pool_ids[i], 'GW'))
        else: 
            args.append((pool_ids[i], 'bruteforce'))
    #print(args)
    with Pool(cores) as pool:
        processed_data = pool.starmap(test, args)


for i in range(cores):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])

# If the error occured you still can try to get processed_data 



array([[  1,   2,   3,   4,   5],
       [  6,   7,   8,   9,  10],
       [ 11,  12,  13,  14,  15],
       [ 16,  17,  18,  19,  20],
       [ 21,  22,  23,  24,  25],
       [ 26,  27,  28,  29,  30],
       [ 31,  32,  33,  34,  35],
       [ 36,  37,  38,  39,  40],
       [ 41,  42,  43,  44,  45],
       [ 46,  47,  48,  49,  50],
       [ 51,  52,  53,  54,  55],
       [ 56,  57,  58,  59,  60],
       [ 61,  62,  63,  64,  65],
       [ 66,  67,  68,  69,  70],
       [ 71,  72,  73,  74,  75],
       [ 76,  77,  78,  79,  80],
       [ 81,  82,  83,  84,  85],
       [ 86,  87,  88,  89,  90],
       [ 91,  92,  93,  94,  95],
       [ 96,  97,  98,  99, 100],
       [101, 102, 103, 104, 105],
       [106, 107, 108, 109, 110],
       [111, 112, 113, 114, 115],
       [116, 117, 118, 119, 120],
       [121, 122, 123, 124, 125],
       [126, 127, 128, 129, 130],
       [131, 132, 133, 134, 135],
       [136, 137, 138, 139, 140],
       [141, 142, 143, 144, 145],
       [146, 1

{'Id (int)': 56, 'Valid (int)': 1, 'Updated (str)': '2025-08-22 15:15:04', 'Matrix (mat)': array([[ 0.2589306 , -0.02092706,  0.03371292,  0.43384691,  0.50536476,
         0.47600472, -0.13454709,  0.0231787 , -0.02151713,  0.13890174],
       [-0.02092706,  0.48354038, -0.45891305,  0.07037588, -0.06551232,
         0.25470049, -0.06100459, -0.2541291 , -0.15254043, -0.11522846],
       [ 0.03371292, -0.45891305,  0.15287895, -0.1134181 , -0.00158767,
         0.41471161, -0.07581213,  0.17491319,  0.07888006, -0.25763408],
       [ 0.43384691,  0.07037588, -0.1134181 ,  0.0477822 , -0.42216167,
         0.03721176,  0.13779857,  0.27753819,  0.08988863, -0.28751574],
       [ 0.50536476, -0.06551232, -0.00158767, -0.42216167, -0.02616456,
        -0.5905133 , -0.20710472,  0.2780364 ,  0.18796913,  0.09059279],
       [ 0.47600472,  0.25470049,  0.41471161,  0.03721176, -0.5905133 ,
        -0.14948651, -0.33829902,  0.21593487,  0.15723299,  0.20310421],
       [-0.13454709, -0.061

In [ ]:

def train (data : list, trainer : Solver, stepsize: float = 0.1, depth : int = 1, log_data = False): 
    ''' 
    ids : np.ndarray 
        array of ids to train
    solver : str 
        solver with which will be trained 
    preprocessing : str 
    postprocessing : str 
    '''

    #print(f'training ids {ids}')
    #print(data)
    processed_data = []
    for line in data:
        #print(type(line))
        Q = line['Matrix (mat)']

        trained_data = line.copy()
        
        train, train_bitstring, train_state, train_iterations, train_itererr, train_time, baren_flag, baren_index = trainer.solve(Q = Q, depth = depth, sol = line['Test (flt)'], stepsize = stepsize, log = False)
        print(f'Trained id = {line['Id (int)']} with train {train}')
        trained_data['Train (flt)'] = train
        trained_data['Train_iterations (int)'] = train_iterations
        trained_data['Train_state (vec)'] = train_bitstring
        for i in range(1, 4): trained_data[f'Train_iterations_{i} (int)'] = train_itererr[i - 1]
        trained_data['Train_time (flt)'] = train_time
        
        current_datetime = datetime.now()
        trained_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
        
        if (baren_flag == True):
            trained_data['Flags (str)'] = f'Baren_{baren_index}'
        trained_data['Train_shots (int)'] = trainer.dev.shots.total_shots
        trained_data['Train_solver (str)'] = trainer.ansatz
        trained_data['Preprocessing (str)'] = trainer.preprocessing 
        trained_data['Postprocessing (str)'] = trainer.postprocessing
        if (log_data):
            log.log(trained_data, byid = True, Id = id)
        processed_data.append(trained_data)
    return processed_data

In [19]:
from multiprocessing import Pool
import pandas as pd 
import pennylane as qml 

cores = 35

solvers = [] 

depth = 3

template = pd.read_csv('data/template.csv')

for i in range (cores):
    dev = qml.device ('lightning.qubit', wires = Config['size'], shots = Config['shots'])
    solvers.append(Solver(dev = dev, ansatz = 'MA-PIA', gate = 'X', iterations_limit = Config['iterations_limit'], accepterr = Config['accepterr'], baren_check = True, baren_rerr = Config['baren_err']))


log = Logger(log_template = template, log_path = source_name.format(size = Config['size'], stiffs = Config['stiffs'], stepsize = Config['stepsize'], shots = Config['shots']))
train_dataset, _ = log._load_logfile()
#print(train_dataset)
data = [[log.read(id) for id in range (id_start, id_start + len(train_dataset) // cores)] for id_start in range (1, len(train_dataset), len(train_dataset) // cores) ]

#print(data)
if (__name__ == '__main__') and (len(train_dataset) % cores):
    
    args = [(data[i], solvers[i], Config['stepsize'], depth) for i in range (cores)]
    #print(train_dataset)
    #print(args)
    with Pool(cores) as pool:
        processed_data = pool.starmap(train, args)


for i in range(cores):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])




ValueError: too many values to unpack (expected 7)

In [ ]:
display(processed_data[3]['Id (int)'])
display(processed_data[3]['Train (flt)'])

arr = [1,2,3,4,5,6,7,8,9,10]
arr = np.array(arr)
arr = arr.reshape(len(arr) // 2, 2)

print(arr)
train(range(1,21), trainer = solvers[0], progressbar = True)
